In [1]:
import requests                                                                   
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from string import punctuation 
from nltk.corpus import stopwords,wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer
import nltk
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [142]:
header={
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    
}

In [144]:
lst=['uppvr','uifd8','ukixi','umkq6','uiogi','ugpv8','ub1t7','ubzhe','tzjke','u7yqt','u3krp','tyirr','tpk7k','tn7n4','tpazf','tn6lq','tkks6','th0nz','ta4zf','tc0rx']
for i in lst:
    url=f"https://gateway.reddit.com/desktopapi/v1/subreddits/Anxiety?rtj=only&redditWebClient=web2x&app=web2x-client-production&allow_over18=&include=prefsSubreddit&after=t3_p{i}&dist=27&forceGeopopular=false&layout=card&sort=hot"
    r=requests.get(url,headers=header)
    text=r.text
    file=open(f'reddit_file_{lst.index(i)+1}.txt','w')
    file.write(text)
    file.close()
    

In [149]:
content=[]
for k in range(1,21):
    file = open(f"reddit_file_{k}.txt","r")
    data=file.read()
    file.close()
    ert=data.split('https://www.reddit.com/r/Anxiety/comments/')
    ert=ert[1:]
    for i in ert:
        qw=i[7:(i.index('"'))-1]
        qw=qw.replace('_',' ')+' .'
        abc=''
        try:
            abc=i[i.index('document": [{"c": [{"e": "text", "t": ')+39 : i.index('"}], "e": "par"}]}, "mediaMetadata":')]
            abc=abc.replace(abc[abc.index('"}'):abc.index('t": ')+4],' ')
            abc=abc.replace(abc[abc.index('", "f": [['):abc.index('ext", "t": "')+12],' ')
        except:
            pass
        content+=[qw+abc]
    
        

In [146]:
fint=pd.DataFrame(content)

In [147]:
fint.to_excel('reddit_file.xlsx')

In [2]:
df=pd.read_excel('reddit_file.xlsx')

In [3]:
train=df[:200]
test=df[200:]

In [5]:
lemma=WordNetLemmatizer()
stemmer=PorterStemmer()


In [8]:
def text_process(t):
    all_words=[]
    t=t.replace('.',' . ').replace('!',' ! ').replace('?',' ? ').replace('/',' / ')
    for sent in nltk.sent_tokenize(t.lower()):
        words=nltk.word_tokenize(sent)
        words=[word for word in words if word not in stopwords.words('english') ]
        words=[stemmer.stem(word) for word in words]
        words=[lemma.lemmatize(word,wordnet.ADJ)for word in words]
        words=[word for word in words if word not in punctuation]
        words=[word for word in words if not word.isnumeric()]
        all_words+=words
    all_words=''.join([ch for ch in ' '.join(all_words) if (ch not in punctuation) and(not ch.isnumeric())])
    return all_words

In [9]:
col=[]
tfidfV = TfidfVectorizer()
tf_idf_train = tfidfV.fit_transform(train['sentences'].apply(text_process))
tf_train = pd.DataFrame(tf_idf_train.toarray(),columns = tfidfV.get_feature_names())
gh=tfidfV.get_feature_names()
tf_idf_test=tfidfV.fit_transform(test['sentences'].apply(text_process))
tf_test = pd.DataFrame(tf_idf_test.toarray(),columns = tfidfV.get_feature_names())
jk=tfidfV.get_feature_names()
for i in gh:
    if i in jk:
        col+=[i]





In [12]:
tf_test.drop(tf_test.columns.difference(col), 1, inplace=True)
tf_train.drop(tf_train.columns.difference(col), 1, inplace=True)

In [15]:
model_bnb = BernoulliNB().fit(tf_train,train['anxious'])
print(model_bnb.score(tf_train,train['anxious']))

0.96


In [16]:
model_mnb = MultinomialNB().fit(tf_train,train['anxious'])
print(model_mnb.score(tf_train,train['anxious']))

0.855


In [17]:
predicted=model_bnb.predict(tf_test)

In [32]:
pred_df=pd.DataFrame({'anxious':predicted,'sentences':test['sentences']})
pred_df.reset_index(inplace = True, drop = True)

In [33]:
pred_df

,anxious,sentences
0,1,just left the doctor and was prescribed .Just ...
1,1,trouble with my anxiety .Lately I\u2019ve been...
2,1,can anyone relate .I can never relax because i...
3,1,elevated h1c .Had my first h1c test this week....
4,0,very anxious about meeting my gfs parents .Hel...
...,...,...
295,1,my anxiety dropped me .Every day seems like an...
296,0,i really want to meet my online friend but i m...
297,0,why thinking of space or other galaxies is giv...
298,1,covid vaccinerelated choking attack .Ever sinc...


In [35]:
pred_df.to_excel('reddit_anxiety_predicted.xlsx')